# <div style="font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:100%; text-align:center;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0">Santa 2023 - dwalton76's NxNxN Cube Solver</div>
## <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:100%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >TABLE OF CONTENTS<br><div>
* [INFO](#0)
* [IMPORTS](#1)
* [LOAD DATA](#2)
* [FUNCTIONS](#3)
  * [FUNCTIONS FROM CRODOC](#3.1)
* [SOLVE](#5)
  * [SCORE](#5.1)
  * [SUBMISSION](#5.2)

<a id="0"></a>
## <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:100%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >INFO<br><div> 
    
Some research online led me to GitHub user [dwalton76's rubiks-cube-NxNxN-solver repo](https://github.com/dwalton76/rubiks-cube-NxNxN-solver). I have implemented the NxNxN cube solver below. Note that the solver will skip cubes with a solution state like "N1;N2;N3;..." or "A;B;A;...". The trick I used on the 3x3x3 cubes causes issues when N>=4 (more to come).
    
Special thanks to @wrrosa's Kociemba notebook. This notebook contains modifications of code found [here](https://www.kaggle.com/code/wrrosa/santa-2023-kociemba-s-two-phase-algo-1-116-550)

<a id="1"></a>
## <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:100%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >IMPORTS<br><div> 
    


In [27]:
%%bash
git clone https://github.com/dwalton76/rubiks-cube-NxNxN-solver.git
cd rubiks-cube-NxNxN-solver
make init

Cloning into 'rubiks-cube-NxNxN-solver'...


rm -rf build dist venv rubikscubennnsolver.egg-info cache ida_search ida_search_via_graph my-pt-states.txt
find . -name __pycache__ | xargs rm -rf
gcc -O3 -o ida_search_via_graph rubikscubennnsolver/ida_search_core.c rubikscubennnsolver/rotate_xxx.c rubikscubennnsolver/ida_search_666.c rubikscubennnsolver/ida_search_777.c rubikscubennnsolver/ida_search_via_graph.c -lm
python3 -m venv venv
  Using cached pip-23.3.2-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1
Obtaining file:///kaggle/working/rubiks-cube-NxNxN-solver/rubiks-cube-NxNxN-solver (from -r requirements.dev.txt (line 10))
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build wheel: starte

In [28]:
%%bash
cd ..
git clone https://github.com/dwalton76/kociemba.git
cd kociemba/kociemba/ckociemba/
make
sudo make install

Cloning into 'kociemba'...


mkdir -p bin
gcc -std=c99 -O3 -D_XOPEN_SOURCE=700 coordcube.c cubiecube.c facecube.c search.c -Iinclude solve.c -o bin/kociemba
cp bin/kociemba /usr/local/bin/


In [1]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import os


<a id="2"></a>
## <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:100%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >LOAD DATA<br><div> 
    
It is becoming difficult to track down the source of some solutions due to all of the ensemble notebooks in use. I've attempted to locate the sources for the files that give the best solution. The Visualize Using Pythreejs appears to use some solutions from a previous version of this notebook: https://www.kaggle.com/code/zaburo/iterative-replacement-of-k-successive-moves
    
Ensemble sources:

-https://www.kaggle.com/code/yunsuxiaozi/k-step-optimal-solution-santa
-https://www.kaggle.com/code/dinhttrandrise/bidirectional-brute-force-w-wildcards
-https://www.kaggle.com/code/jazivxt/visualize-using-pythreejs
-https://www.kaggle.com/code/seanbearden/kociemba-w-greedy-all-3x3x3-ensemble

In [2]:
sol1 = pd.read_csv('kstep.csv')
sol2 = pd.read_csv('bidirect.csv')
sol3 = pd.read_csv('viz.csv')
sol4 = pd.read_csv('kociemba.csv')


ens = pd.DataFrame({
    'solution1': sol1['moves'],
    'solution2': sol2['moves'],
    'solution3': sol3['moves'],
    'solution4': sol4['moves'],
})

In [3]:
ens

,solution1,solution2,solution3,solution4
0,r1.-f1,r1.-f1,r1.-f1,r1.-f1
1,f0.r1.f1.-d0.-d0.-f0.-r0.f0.d0,f0.r1.f1.-d0.-d0.-f0.-r0.f0.d0,f0.r1.f1.-d0.-d0.-f0.-r0.f0.d0,f0.r1.f1.-d0.-d0.-f0.-r0.f0.d0
2,-d1.-r0.f0.-r1.f1.d1.-r1.-f0.d1.f0.d1.d1,-d1.-r0.f0.-r1.f1.d1.-r1.-f0.d1.f0.d1.d1,-d1.-r0.f0.-r1.f1.d1.-r1.-f0.d1.f0.d1.d1,-d1.-r0.f0.-r1.f1.d1.-r1.-f0.d1.f0.d1.d1
3,-f0.d0.-r0.f0.-d0.-r0.d0.-f0.-r0.-f0,-f0.d0.-r0.f0.-d0.-r0.d0.-f0.-r0.-f0,-f0.d0.-r0.f0.-d0.-r0.d0.-f0.-r0.-f0,-f0.d0.-r0.f0.-d0.-r0.d0.-f0.-r0.-f0
4,-r1.-f0.d0.r0.-d1.-d1.r1.d1.f0.r1.-d1.-r1,-r1.-f0.d0.r0.-d1.-d1.r1.d1.f0.r1.-d1.-r1,-r1.-f0.d0.r0.-d1.-d1.r1.d1.f0.r1.-d1.-r1,-r1.-f0.d0.r0.-d1.-d1.r1.d1.f0.r1.-d1.-r1
...,...,...,...,...
393,f19.f21.-f39.f20.f2.-f5.f7.-r3.f55.-f12.f65.-f...,f19.f21.-f39.f20.f2.-f5.f7.-r3.f55.-f12.f65.-f...,f19.f21.-f39.f20.f2.-f5.f7.-r3.f55.-f12.f65.-f...,f19.f21.-f39.f20.f2.-f5.f7.-r3.f55.-f12.f65.-f...
394,-f31.-f22.f16.-f17.-f13.-f24.-f14.f2.f21.f44.f...,-f31.-f22.f16.-f17.-f13.-f24.-f14.f2.f21.f44.f...,-f31.-f22.f16.-f17.-f13.-f24.-f14.f2.f21.f44.f...,-f31.-f22.f16.-f17.-f13.-f24.-f14.f2.f21.f44.f...
395,-r0.-f42.-f8.f16.-f49.f14.-f1.f56.f26.f35.f62....,-r0.-f42.-f8.f16.-f49.f14.-f1.f56.f26.f35.f62....,-r0.-f42.-f8.f16.-f49.f14.-f1.f56.f26.f35.f62....,-r0.-f42.-f8.f16.-f49.f14.-f1.f56.f26.f35.f62....
396,f25.-f29.f46.f49.-f8.f27.f26.-f20.f2.-f20.f6.f...,f25.-f29.f46.f49.-f8.f27.f26.-f20.f2.-f20.f6.f...,f25.-f29.f46.f49.-f8.f27.f26.-f20.f2.-f20.f6.f...,f25.-f29.f46.f49.-f8.f27.f26.-f20.f2.-f20.f6.f...


In [4]:
ens.apply(lambda moves: min(moves, key=lambda x: len(x.split('.'))), axis=1)

0                                                 r1.-f1
1                         f0.r1.f1.-d0.-d0.-f0.-r0.f0.d0
2               -d1.-r0.f0.-r1.f1.d1.-r1.-f0.d1.f0.d1.d1
3                   -f0.d0.-r0.f0.-d0.-r0.d0.-f0.-r0.-f0
4              -r1.-f0.d0.r0.-d1.-d1.r1.d1.f0.r1.-d1.-r1
                             ...                        
393    f19.f21.-f39.f20.f2.-f5.f7.-r3.f55.-f12.f65.-f...
394    -f31.-f22.f16.-f17.-f13.-f24.-f14.f2.f21.f44.f...
395    -r0.-f42.-f8.f16.-f49.f14.-f1.f56.f26.f35.f62....
396    f25.-f29.f46.f49.-f8.f27.f26.-f20.f2.-f20.f6.f...
397    -r1.f23.f24.-r2.f38.-f19.f24.f19.f18.f29.f48.-...
Length: 398, dtype: object

In [12]:
puzzle_info

,puzzle_type,allowed_moves
0,cube_2/2/2,"{'f0': [0, 1, 19, 17, 6, 4, 7, 5, 2, 9, 3, 11,..."
1,cube_3/3/3,"{'f0': [0, 1, 2, 3, 4, 5, 44, 41, 38, 15, 12, ..."
2,cube_4/4/4,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
3,cube_5/5/5,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
4,cube_6/6/6,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
5,cube_7/7/7,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
6,cube_8/8/8,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
7,cube_9/9/9,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
8,cube_10/10/10,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."
9,cube_19/19/19,"{'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ..."


In [13]:
ens['ens_solution'] = ens.apply(lambda moves: min(moves, key=lambda x: len(x.split('.'))), axis=1)
ensemble_score = ens['ens_solution'].str.split('.').apply(lambda x: len(x)).sum()


puzzles = pd.read_csv('puzzles.csv', index_col='id')
puzzle_info = pd.read_csv( 'puzzle_info.csv')
submission = pd.read_csv('sample_submission.csv', index_col='id')

submission['moves'] = ens['ens_solution']

In [14]:
submission

,moves
id,
0,r1.-f1
1,f0.r1.f1.-d0.-d0.-f0.-r0.f0.d0
2,-d1.-r0.f0.-r1.f1.d1.-r1.-f0.d1.f0.d1.d1
3,-f0.d0.-r0.f0.-d0.-r0.d0.-f0.-r0.-f0
4,-r1.-f0.d0.r0.-d1.-d1.r1.d1.f0.r1.-d1.-r1
...,...
393,f19.f21.-f39.f20.f2.-f5.f7.-r3.f55.-f12.f65.-f...
394,-f31.-f22.f16.-f17.-f13.-f24.-f14.f2.f21.f44.f...
395,-r0.-f42.-f8.f16.-f49.f14.-f1.f56.f26.f35.f62....


<a id="3"></a>
## <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:100%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >FUNCTIONS<br><div> 

In [15]:
U = ['U', 'F', 'R', 'B', 'L', 'D']
U_dict = {
    'A': 'U',
    'B': 'F',
    'C': 'R',
    'D': 'B',
    'E': 'L',
    'F': 'D',
}
def state2ubl(state):
    state_split = state.split(';')
    dim = int(np.sqrt(len(state_split) // 6))
    dim_2 = dim**2
    
    s = ''.join([U_dict[f] for f in state_split])
    
    return s[:dim_2] + s[2*dim_2:3*dim_2] + s[dim_2:2*dim_2] + s[5*dim_2:] + s[4*dim_2:5*dim_2] + s[3*dim_2:4*dim_2]


In [16]:
puzzles.iloc[1]['solution_state']

'A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F'

In [17]:
state2ubl('A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F')

'UUUURRRRFFFFDDDDLLLLBBBB'

In [18]:
def move_translation(dim):
    M = {}
    M["U"] = f'-d{dim-1}'
    M["R"] = "r0"
    M["B"] = f"-f{dim-1}"
    M["F"] = "f0"
    M["L"] = f"-r{dim-1}"
    M["D"] = "d0"

    if dim > 3:
        M["Uw"] = f'-d{dim-2}.-d{dim-1}'
        M["Rw"] = f"r0.r1"
        M["Bw"] = f'-f{dim-2}.-f{dim-1}'
        M["Fw"] = f"f0.f1"
        M["Lw"] = f'-r{dim-2}.-r{dim-1}'
        M["Dw"] = f"d0.d1"
        
    if dim >= 6:
        M["2Uw"] = f'-d{dim-2}.-d{dim-1}'
        M["2Rw"] = f"r0.r1"
        M["2Bw"] = f'-f{dim-2}.-f{dim-1}'
        M["2Fw"] = f"f0.f1"
        M["2Lw"] = f'-r{dim-2}.-r{dim-1}'
        M["2Dw"] = f"d0.d1"

        width_max = dim // 2
        for i in range(3, width_max + 1):
            M[f"{i}Uw"] = f'-d{dim-i}.' + M[f"{i-1}Uw"]
            M[f"{i}Rw"] = M[f"{i-1}Rw"] + f'.r{i-1}'
            M[f"{i}Bw"] = f'-f{dim-i}.' + M[f"{i-1}Bw"]
            M[f"{i}Fw"] = M[f"{i-1}Fw"] + f'.f{i-1}'
            M[f"{i}Lw"] = f'-r{dim-i}.' + M[f"{i-1}Lw"]
            M[f"{i}Dw"] = M[f"{i-1}Dw"] + f'.d{i-1}'


    for m in list(M):
        M[m+"2"] = M[m] + '.' + M[m]
        if "-" in M[m]:
            M[m+"'"] = M[m].replace("-","")
        else:
            M[m+"'"] = '.'.join(["-"+i for i in M[m].split('.')])
    
    return M

In [19]:
move_translation(6)

{'U': '-d5',
 'R': 'r0',
 'B': '-f5',
 'F': 'f0',
 'L': '-r5',
 'D': 'd0',
 'Uw': '-d4.-d5',
 'Rw': 'r0.r1',
 'Bw': '-f4.-f5',
 'Fw': 'f0.f1',
 'Lw': '-r4.-r5',
 'Dw': 'd0.d1',
 '2Uw': '-d4.-d5',
 '2Rw': 'r0.r1',
 '2Bw': '-f4.-f5',
 '2Fw': 'f0.f1',
 '2Lw': '-r4.-r5',
 '2Dw': 'd0.d1',
 '3Uw': '-d3.-d4.-d5',
 '3Rw': 'r0.r1.r2',
 '3Bw': '-f3.-f4.-f5',
 '3Fw': 'f0.f1.f2',
 '3Lw': '-r3.-r4.-r5',
 '3Dw': 'd0.d1.d2',
 'U2': '-d5.-d5',
 "U'": 'd5',
 'R2': 'r0.r0',
 "R'": '-r0',
 'B2': '-f5.-f5',
 "B'": 'f5',
 'F2': 'f0.f0',
 "F'": '-f0',
 'L2': '-r5.-r5',
 "L'": 'r5',
 'D2': 'd0.d0',
 "D'": '-d0',
 'Uw2': '-d4.-d5.-d4.-d5',
 "Uw'": 'd4.d5',
 'Rw2': 'r0.r1.r0.r1',
 "Rw'": '-r0.-r1',
 'Bw2': '-f4.-f5.-f4.-f5',
 "Bw'": 'f4.f5',
 'Fw2': 'f0.f1.f0.f1',
 "Fw'": '-f0.-f1',
 'Lw2': '-r4.-r5.-r4.-r5',
 "Lw'": 'r4.r5',
 'Dw2': 'd0.d1.d0.d1',
 "Dw'": '-d0.-d1',
 '2Uw2': '-d4.-d5.-d4.-d5',
 "2Uw'": 'd4.d5',
 '2Rw2': 'r0.r1.r0.r1',
 "2Rw'": '-r0.-r1',
 '2Bw2': '-f4.-f5.-f4.-f5',
 "2Bw'": 'f4.f5',
 '2Fw2': 

<a id="3.1"></a>
## <div style="font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:#6A1B9A; font-size:100%; text-align:left; padding:3.0px; background: white; border-bottom: 8px solid #9C27B0">FUNCTIONS FROM CRODOC<br><div>

* https://www.kaggle.com/code/crodoc/1-187-898-greedy-baseline-improvement

In [20]:
allowed_moves = {}

for puzzle_type, row in puzzle_info.iterrows():
    allowed_moves[puzzle_type] = eval(row['allowed_moves'])

In [21]:
allowed_moves = {}

for puzzle_type, row in puzzle_info.iterrows():
    allowed_moves[puzzle_type] = eval(row['allowed_moves'])

def move_state(state, move, moves_pool):
    
    if '-' in move:
        move = move[1:]
        rev = True
    else:
        rev = False

    move = moves_pool[move]

    if rev:
        new_state = state[:]
        for i in range(len(move)):
            new_state[move[i]] = state[i]
        state = new_state
    else:
        state = [state[idx] for idx in move]
    
    return state

def solve_puzzle(current_puzzle, moves_str):
    # paths.loc[paths['id'] == puzzle_id]
    # todo: assumes index aligned with puzzle id. Need to update.
#     current_puzzle = path.loc[puzzle_id]
    puzzle_type = current_puzzle['puzzle_type']
    initial_state = current_puzzle['initial_state']
    solution_state = current_puzzle['solution_state']
    
#     moves = submission_df.loc[puzzle_id]['moves'].split('.')
    moves = moves_str.split('.')
    moves_pool = allowed_moves[puzzle_type]
    
    state = initial_state.split(';')
    
    state_list = []
    state_list.append(state)
    
    for move in moves:
        state = move_state(state, move, moves_pool)
        state_list.append(state)
    
    state_to_idx = {}
    for idx in range(len(state_list)):
        state_to_idx[';'.join(state_list[idx])] = idx
    
    res = []
    idx = 0
    
    # ignore last state
    for curr_idx in tqdm(list(range(len(state_list)-1))):
        
        if curr_idx != idx:
            continue
        
        state = state_list[idx]
        
        new_idx = -1
        new_move = ''
        
        for move in moves_pool:
            for reversed_move in ['', '-']:
                move = reversed_move + move
                
                new_state = ';'.join(move_state(state, move, moves_pool))
                
                if new_state in state_to_idx:
                    tmp_idx = state_to_idx[new_state]

                    if tmp_idx > new_idx:
                        new_idx = tmp_idx
                        new_move = move
        
        idx = new_idx
        res.append(new_move)
            
    print('MOVES BEFORE:', len(state_list)-1)
    print('MOVES AFTER:', len(res))
    print()
    
    return '.'.join(res)

<a id="5"></a>
## <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:100%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >SOLVE<br><div> 
    
Solver will skip cubes with a solution state like "N1;N2;N3;..." or "A;B;A;...".

In [22]:
GREEDY_REDUCTION = False

outputs = {}
for id, row in puzzles.iterrows():
    if row['puzzle_type'][:4] == 'cube':
        
        dim = int(row['puzzle_type'].split('/')[-1])
        if id not in range(30, 284):
            continue
        moves = eval(puzzle_info.loc[row['puzzle_type'], 'allowed_moves'])
        for move in list(moves):
            moves['-'+move] = np.argsort(moves[move]).tolist()
        
        M = move_translation(dim)

        init_state = row['initial_state']
        sol_state = row['solution_state']

        # Only attempt to solve cubes with traditional solutions. 
        # That is, cubes that have a solution state where each face 
        # filled with equivilent letters/colors.
        if sol_state[:2*dim**2-1] == ';'.join(['A']*dim**2):
            state = state2ubl(init_state)
            print(f'Starting {id}')
            output = !./rubiks-cube-solver.py --state $state
            outputs[id] = output
            if output[-1][:9] == 'Solution:':
                sol = output[-1].split(': ')[1]
            else:
                for n in range(1, 21):
                    if 'Solution:' in output[-n]:
                        sol = output[-n].split('Solution: ')[1].split('2023-')[0]
                        break
                
            mmoves = '.'.join([M[m] for m in sol.split(' ')])
            
            new_state = init_state
            for move in mmoves.split('.'):
                new_state = ';'.join(list(np.asarray(new_state.split(';'))[np.array(moves[move])]))
            
            I = ['.'.join([f'{j}{i}' for i in range(dim)]) for j in ['r', 'd', 'f']]
            manipulations = [''] + I + [i1 + '.' + i2 for i1 in I for i2 in I]+ [i1 + '.' + i2+ '.' + i3 for i1 in I for
                                                                              i2 in I for i3 in I]+ [i1 + '.' + i2+ '.' + i3 + '.' + i4 for i1 in I for i2 in I for i3 in I for i4 in I]
            for init_moves in manipulations:
                temp_state = new_state
                if len(init_moves) > 0:
                    for move in init_moves.split('.'):
                        temp_state = ';'.join(list(np.asarray(temp_state.split(';'))[np.array(moves[move])]))
                if temp_state == sol_state:
                    print(f'solved id: {id}')
                    if len(init_moves) > 0:
                        mmoves += '.' + init_moves
                    break
                    
            # validation
            state = row['initial_state'].split(";")
            for move_name in mmoves.split('.'):
                state = [state[i] for i in moves[move_name]]

            try:
                
                assert row['solution_state'].split(";") == state
                
                if GREEDY_REDUCTION:
                    # reduce moves
                    mmoves = solve_puzzle(puzzles.loc[id], mmoves)


                mmoves_length = len(mmoves.split('.'))
                best_moves_length = len(submission.loc[id, 'moves'].split('.')) 
                
                if mmoves_length < best_moves_length:
                    # Insert the moves into the database
                    submission.loc[id, 'moves'] = mmoves
            except AssertionError:
                print(f"assertion error for {id}")
                print(row['solution_state'])
                print(';'.join(state))
                break


KeyError: 'cube_3/3/3'

<a id="5.1"></a>
## <div style="font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:#6A1B9A; font-size:100%; text-align:left; padding:3.0px; background: white; border-bottom: 8px solid #9C27B0">SCORE<br><div>

In [ ]:
print(f"Score: {submission['moves'].str.split('.').apply(lambda x: len(x)).sum()}")

<a id="5.2"></a>
## <div style="font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:#6A1B9A; font-size:100%; text-align:left; padding:3.0px; background: white; border-bottom: 8px solid #9C27B0">SUBMISSION<br><div>

In [ ]:
submission.reset_index().to_csv('submission.csv', index=False)

os.chdir('..')
print("Current Working Directory: ", os.getcwd())

submission.reset_index().to_csv('submission.csv', index=False)

FYI: There are methods available to reduce some of these solutions further.